In [1]:
%pip install bertopic --user 

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install cuda-python --user

  Obtaining dependency information for cuda-python from https://files.pythonhosted.org/packages/03/80/b42e8fbae7fdaa21e4651e90cc21f69239047b7e6b957b988e0dfc3c9679/cuda_python-12.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 47.6 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from bertopic import BERTopic
import pandas as pd
import pickle

In [2]:
# Since BERT uses sentence embeddings in it's process we want the stop words but the removed ones are useless
# Testing on this one but it's really small so idk if the topics will be good
print(f"Reading in data")
dfs = []
# pd.read_json("C:\\Users\\russe\\OneDrive\\Documents\\git\\Optus-data\\drugs-forum\\hydromorphone.json", orient='records')
dfs.append(pd.read_json("data/buprenorphine.json", orient='records'))
dfs.append(pd.read_json("data/codeine.json", orient='records'))
dfs.append(pd.read_json("data/heroin.json", orient='records'))
dfs.append(pd.read_json("data/hydrocodone.json", orient='records'))
dfs.append(pd.read_json("data/hydromorphone.json", orient='records'))
dfs.append(pd.read_json("data/methadone.json", orient='records'))
dfs.append(pd.read_json("data/morphine.json", orient='records'))
dfs.append(pd.read_json("data/opium-poppy.json", orient='records'))
dfs.append(pd.read_json("data/oxycodone.json", orient='records'))
dfs.append(pd.read_json("data/oxymorphone.json", orient='records'))
dfs.append(pd.read_json("data/tramadol.json", orient='records'))
# test = pd.DataFrame.from_dict(json.load(open("C:\\Users\\russe\\OneDrive\\Documents\\git\\Optus-data\\drugs-forum\\hydrocodone.json", encoding='utf-8')))

for i in range(len(dfs)):
    print(f"File {i} shape: {dfs[i].shape}")



Reading in data
File 0 shape: (9406, 10)
File 1 shape: (11135, 10)
File 2 shape: (48437, 10)
File 3 shape: (5215, 10)
File 4 shape: (2906, 10)
File 5 shape: (9030, 10)
File 6 shape: (5132, 10)
File 7 shape: (11630, 10)
File 8 shape: (15932, 10)
File 9 shape: (1430, 10)
File 10 shape: (6797, 10)


In [3]:
print(f"Combining data")
df = pd.concat(dfs)
df

Combining data


,username,post_content,rank,rep_points,messages,join_date,country_of_origin,date,post_title,post_type
0,DanDone,Hi. I have a small codeine phosphate habit - I...,Newbie,10,8,1458100800000,from U.K.,2023-10-18 04:00:00,Buprenorphine patch for codeine withdrawals,Question
1,TheBigBadWolf,Hallo @Dan.\nTrying to get yourself straight a...,Gold Member,"16,967","9,563",1270958400000,from Germany,2023-10-18 04:00:00,Buprenorphine patch for codeine withdrawals,Question
2,DanDone,Thank you. I do have some pregabalin so will c...,Newbie,10,8,1458100800000,from U.K.,2023-10-18 04:00:00,Buprenorphine patch for codeine withdrawals,Question
3,TheBigBadWolf,That's a good decision. \nAs I Had Said bupren...,Gold Member,"16,967","9,563",1270958400000,from Germany,2023-10-18 04:00:00,Buprenorphine patch for codeine withdrawals,Question
4,Archangel Zadkiel,Thanks for the shout out @TheBigBadWolf long t...,Palladium Member,"3,445","1,936",1333425600000,"44\ny/o\nfrom Cincinnati, Ohio",2023-10-19 04:00:00,Buprenorphine patch for codeine withdrawals,Question
...,...,...,...,...,...,...,...,...,...,...
6792,paau,"Ok so I took 200mg of tramadol last wednesday,...",Newbie,14,40,1186459200000,35\ny/o\nfrom U.S.A.,2007-10-09 04:00:00,\nEffects of Tramadol when trying to come up o...,
6793,trptamene,If the rolls are known to be good this WAS the...,Platinum Member & Advisor,"2,009","1,677",1121313600000,from U.S.A.,2007-10-09 04:00:00,\nEffects of Tramadol when trying to come up o...,
6794,paau,"yeah sucks a bit really, I hasn't got much tra...",Newbie,14,40,1186459200000,35\ny/o\nfrom U.S.A.,2007-10-11 04:00:00,\nEffects of Tramadol when trying to come up o...,
6795,Nagognog2,Many people prescribed tramadol (Ultram) are n...,Iridium Member,"1,936","6,713",1107234000000,None,2007-10-11 04:00:00,\nEffects of Tramadol when trying to come up o...,


In [4]:
print("removing not-needed data")
df = df.loc[:, ['username','post_content']]
df

removing not-needed data


,username,post_content
0,DanDone,Hi. I have a small codeine phosphate habit - I...
1,TheBigBadWolf,Hallo @Dan.\nTrying to get yourself straight a...
2,DanDone,Thank you. I do have some pregabalin so will c...
3,TheBigBadWolf,That's a good decision. \nAs I Had Said bupren...
4,Archangel Zadkiel,Thanks for the shout out @TheBigBadWolf long t...
...,...,...
6792,paau,"Ok so I took 200mg of tramadol last wednesday,..."
6793,trptamene,If the rolls are known to be good this WAS the...
6794,paau,"yeah sucks a bit really, I hasn't got much tra..."
6795,Nagognog2,Many people prescribed tramadol (Ultram) are n...


In [5]:
# sentence embeddings
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda') # fast but pretty accurate embeddings


In [6]:
embeddings = embedding_model.encode(df['post_content'].to_list(), show_progress_bar=True)

with open("embeddings.pkl", "wb") as fOut:
    pickle.dump({'embeddings-only1': embeddings},fOut)

RuntimeError: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

In [8]:
%dir

UsageError: Line magic function `%dir` not found.
